In [1]:
import os
import glob
import inro.modeller as m
import inro.emme.matrix as _matrix
import numpy as np
import pandas as pd
mm = m.Modeller()    
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
eb = mm.emmebank

In [2]:
scen_bau = 404

In [3]:
### IMPORT ALL MATRICES IN SUBFOLDERS ###
def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)
def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"

In [4]:
#Truck matrices
#Truck matrices
truck_l_demand_am = "am_light_2051_20191227"
truck_m_demand_am = "am_medium_2051_20191227"
truck_h_demand_am = "am_heavy_2051_20191227"
truck_l_demand_md = "md_light_2051_20191227"
truck_m_demand_md = "md_medium_2051_20191227"
truck_h_demand_md = "md_heavy_2051_20191227"
truck_l_demand_pm = "pm_light_2051_20191227"
truck_m_demand_pm = "pm_medium_2051_20191227"
truck_h_demand_pm = "pm_heavy_2051_20191227"
truck_l_demand_ev = "ev_light_2051_20191227"
truck_m_demand_ev = "ev_medium_2051_20191227"
truck_h_demand_ev = "ev_heavy_2051_20191227"
truck_l_demand_dn = "dn_light_2051_20191227"
truck_m_demand_dn = "dn_medium_2051_20191227"
truck_h_demand_dn = "dn_heavy_2051_20191227"

truck_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Long List\BAU\MTO_2051_GGH_MTP_5002_V106A3_HS032_20200117\Assignable matrices"

truck_list = {truck_h_demand_am: '',
            truck_l_demand_am: '',
            truck_m_demand_am: '',
            truck_h_demand_md: '',
            truck_l_demand_md: '',
            truck_m_demand_md: '',
            truck_h_demand_pm: '',
            truck_l_demand_pm: '',
            truck_m_demand_pm: '',
            truck_h_demand_ev: '',
            truck_l_demand_ev: '',
            truck_m_demand_ev: '',
            truck_h_demand_dn: '',
            truck_l_demand_dn: '',
            truck_m_demand_dn: '',}

for matrix_name in truck_list.keys():
    truck_list[matrix_name] = eb.available_matrix_identifier('FULL')
    print matrix_name
    create_matrix(matrix_id = truck_list[matrix_name],
                  matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                  matrix_description = matrix_name)

import_matrices_from_directory(truck_folder, truck_list, ".mdf",401) 

am_light_2051_20191227
ev_heavy_2051_20191227
md_medium_2051_20191227
pm_light_2051_20191227
ev_light_2051_20191227
dn_medium_2051_20191227
pm_medium_2051_20191227
pm_heavy_2051_20191227
md_light_2051_20191227
dn_light_2051_20191227
am_medium_2051_20191227
ev_medium_2051_20191227
am_heavy_2051_20191227
md_heavy_2051_20191227
dn_heavy_2051_20191227
Imported matrix mf13
Imported matrix mf1
Imported matrix mf11
Imported matrix mf15
Imported matrix mf10
Imported matrix mf6
Imported matrix mf2
Imported matrix mf5
Imported matrix mf12
Imported matrix mf14
Imported matrix mf9
Imported matrix mf3
Imported matrix mf8
Imported matrix mf4
Imported matrix mf7
Finished Importing Matrices


In [5]:
# Light Trucks - PEAK
light_pkdem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = light_pkdem_truck,
              matrix_name="trips.peak.truck.light",
            matrix_description = "trips.peak.truck.light")
exp = "({} + {})".format(truck_list[truck_l_demand_am],truck_list[truck_l_demand_pm])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": light_pkdem_truck,
    "expression": exp
}
matrix_calculator(spec)

# Medium Trucks - PEAK
med_pkdem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = med_pkdem_truck,
              matrix_name="trips.peak.truck.medium",
            matrix_description = "trips.peak.truck.medium")

exp = "({} + {})".format(truck_list[truck_m_demand_am],truck_list[truck_m_demand_pm])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": med_pkdem_truck,
    "expression": exp
}
matrix_calculator(spec)

# Heavy Trucks - PEAK
heavy_pkdem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = heavy_pkdem_truck,
              matrix_name="trips.peak.truck.heavy",
            matrix_description = "trips.peak.truck.heavy")

exp = "({} + {})".format(truck_list[truck_h_demand_am],truck_list[truck_h_demand_pm])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": heavy_pkdem_truck,
    "expression": exp
}
matrix_calculator(spec)

# Light Trucks - OFFPEAK
light_offpkdem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = light_offpkdem_truck,
              matrix_name="trips.offpeak.truck.light",
            matrix_description = "trips.offpeak.truck.light")
exp = "({} + {} + {})".format(truck_list[truck_l_demand_md],truck_list[truck_l_demand_ev], truck_list[truck_l_demand_dn])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": light_offpkdem_truck,
    "expression": exp
}
matrix_calculator(spec)

# Medium Trucks - OFFPEAK
med_offpkdem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = med_offpkdem_truck,
              matrix_name="trips.offpeak.truck.medium",
            matrix_description = "trips.offpeak.truck.medium")

exp = "({} + {} + {})".format(truck_list[truck_l_demand_md],truck_list[truck_l_demand_ev], truck_list[truck_l_demand_dn])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": med_offpkdem_truck,
    "expression": exp
}
matrix_calculator(spec)

# Heavy Trucks - OFFPEAK
heavy_offpkdem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = heavy_offpkdem_truck,
              matrix_name="trips.offpeak.truck.heavy",
            matrix_description = "trips.offpeak.truck.heavy")

exp = "({} + {} + {})".format(truck_list[truck_l_demand_md],truck_list[truck_l_demand_ev], truck_list[truck_l_demand_dn])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": heavy_offpkdem_truck,
    "expression": exp
}

matrix_calculator(spec)

print "Finished calculating truck demand"

Finished calculating truck demand


In [7]:
# Total Demand
dem_trck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id =dem_trck,
              matrix_name="trips.total",
            matrix_description = "trips.total")

exp = light_pkdem_truck + "+" + med_pkdem_truck + "+" + light_pkdem_truck + "+" + \
      light_offpkdem_truck + "+" + med_offpkdem_truck + "+" + light_offpkdem_truck

spec = {
    "type": "MATRIX_CALCULATION",
    "result": dem_trck,
    "expression": exp
}
matrix_calculator(spec)

{'average': 0.068694,
 'expression_type': 'FULL',
 'maximum': 125.183357,
 'maximum_at': {'destination': 4079, 'origin': 4111},
 'minimum': 0.0,
 'minimum_at': {'destination': 1001, 'origin': 1001},
 'num_evaluations': 12327121,
 'result_type': 'FULL',
 'sum': 846798.8125}

In [8]:
indices = eb.matrix(dem_trck).get_data(scen_bau).indices
mat_np = eb.matrix(dem_trck).get_numpy_data(scen_bau)
df_full = pd.DataFrame(mat_np,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "O",
                                                                                           "level_1": "D",
                                                                                          0: "dem_trck"})

In [10]:
df_full.columns

Index([u'O', u'D', u'dem_trck'], dtype='object')

In [20]:
pth = r"C:\Users\PECHEN\Documents\Sketch Modelling"
megazones_df = pd.read_csv(os.path.join(pth,"TAZ_SketchModelZones_20201110.csv"))
df_int_ext = df_full[((df_full.O <= 9423) & (df_full.D <= 9423))]
df_int_ext = df_int_ext.merge(megazones_df, left_on = "O", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_O"})
df_int_ext = df_int_ext.merge(megazones_df, left_on = "D", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_D"})

In [21]:
df_int_ext.columns

Index([u'O', u'D', u'dem_trck', u'Megazone_O', u'Megazone_D'], dtype='object')

In [22]:
megazone_df = pd.pivot_table(df_int_ext,values = "dem_trck", index = ["Megazone_O","Megazone_D"],aggfunc = sum).reset_index()

In [23]:
df_full.to_csv(os.path.join(pth,"Test.csv"))

In [24]:
megazone_df[(megazone_df.Megazone_O == "Extl West") | (megazone_df.Megazone_O == "Extl North") | (megazone_df.Megazone_O == "Extl South")| (megazone_df.Megazone_O == "Extl East")|(megazone_df.Megazone_D == "Extl West") | (megazone_df.Megazone_D == "Extl North") | (megazone_df.Megazone_D == "Extl South")| (megazone_df.Megazone_D == "Extl East")]["dem_trck"].sum()

5369.5707772245951

In [25]:
megazone_df[(megazone_df.Megazone_O == "Extl West") | (megazone_df.Megazone_O == "Extl North") | (megazone_df.Megazone_O == "Extl South")| (megazone_df.Megazone_O == "Extl East")|(megazone_df.Megazone_D == "Extl West") | (megazone_df.Megazone_D == "Extl North") | (megazone_df.Megazone_D == "Extl South")| (megazone_df.Megazone_D == "Extl East")].to_csv(os.path.join(pth,"Megazone_TrckExternal_OD_2051BAU_20210129.csv"))

In [26]:
megazone_df = megazone_df.pivot(index="Megazone_O", columns="Megazone_D", values = "dem_trck")


In [29]:
megazone_df.to_csv(os.path.join(pth,"TESttt.csv"))